In [1]:
import os
import skimage
from skimage import io, transform
import math
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, concatenate,ZeroPadding2D, BatchNormalization,Concatenate,GlobalAveragePooling2D, Dropout,Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img  
import cv2

%matplotlib inline

import keras.backend as K
K.set_learning_phase(1)

Using TensorFlow backend.


In [24]:
def Conv2D_BN_AC(X, filters, kernel, strides, padding, activation):
    X = Conv2D(filters, kernel, strides=strides, padding=padding)(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation(activation)(X)
    return X

def Stem(X):
    X = Conv2D_BN_AC(X, 32, (3, 3), (2, 2), 'valid', 'relu')
    X = Conv2D_BN_AC(X, 32, (3, 3), (1, 1), 'valid', 'relu')
    X = Conv2D_BN_AC(X, 64, (3, 3), (1, 1), 'same', 'relu')
    a = MaxPooling2D((3, 3), strides=(2, 2))(X)
    b = Conv2D_BN_AC(X, 96, (3, 3), (2, 2), 'valid', 'relu')
    X = concatenate([a,b], axis=3)  
    a = Conv2D_BN_AC(X, 64, (1, 1), (1, 1), 'same', 'relu')
    a = Conv2D_BN_AC(a, 96, (3, 3), (1, 1), 'valid', 'relu')
    b = Conv2D_BN_AC(X, 64, (1, 1), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(b, 64, (7, 1), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(b, 64, (1, 7), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(b, 96, (3, 3), (1, 1), 'valid', 'relu')
    X = concatenate([a,b], axis=3)
    a = Conv2D_BN_AC(X, 192, (3, 3), (2, 2), 'valid', 'relu')
    b = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = concatenate([a,b], axis=3)
    X = Activation('relu')(X)
    return X

def Inception_A(X):
    a = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(X)
    a = Conv2D_BN_AC(a, 96, (1, 1), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(X, 96, (1, 1), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(X, 64, (1, 1), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(c, 96, (3, 3), (1, 1), 'same', 'relu')
    d = Conv2D_BN_AC(X, 64, (1, 1), (1, 1), 'same', 'relu')
    d = Conv2D_BN_AC(d, 96, (3, 3), (1, 1), 'same', 'relu')
    d = Conv2D_BN_AC(d, 96, (3, 3), (1, 1), 'same', 'relu')
    X = concatenate([a,b,c,d], axis=3)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    return X

def Inception_B(X):
    a = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(X)
    a = Conv2D_BN_AC(a, 128, (1, 1), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(X, 384, (1, 1), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(X, 192, (1, 1), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(c, 224, (1, 7), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(c, 256, (7, 1), (1, 1), 'same', 'relu')
    d = Conv2D_BN_AC(X, 192, (1, 1), (1, 1), 'same', 'relu')
    d = Conv2D_BN_AC(d, 192, (1, 7), (1, 1), 'same', 'relu')
    d = Conv2D_BN_AC(d, 224, (7, 1), (1, 1), 'same', 'relu')
    d = Conv2D_BN_AC(d, 224, (1, 7), (1, 1), 'same', 'relu')
    d = Conv2D_BN_AC(d, 256, (7, 1), (1, 1), 'same', 'relu')
    X = concatenate([a,b,c,d], axis=3)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    return X

def Inception_C(X):
    a = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(X)
    a = Conv2D_BN_AC(a, 256, (1, 1), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(X, 256, (1, 1), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(X, 384, (1, 1), (1, 1), 'same', 'relu')
    c1 = Conv2D_BN_AC(c, 256, (1, 3), (1, 1), 'same', 'relu')
    c2 = Conv2D_BN_AC(c, 256, (3, 1), (1, 1), 'same', 'relu')
    d = Conv2D_BN_AC(X, 384, (1, 1), (1, 1), 'same', 'relu')
    d = Conv2D_BN_AC(d, 448, (1, 3), (1, 1), 'same', 'relu')
    d = Conv2D_BN_AC(d, 512, (3, 1), (1, 1), 'same', 'relu')
    d1 = Conv2D_BN_AC(d, 256, (3, 1), (1, 1), 'same', 'relu')
    d2 = Conv2D_BN_AC(d, 256, (1, 3), (1, 1), 'same', 'relu')
    X = concatenate([a,b,c1,c2,d1,d2], axis=3)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    return X
    
def Reduction_A(X):
    a = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(X)
    b = Conv2D_BN_AC(X, 384, (3, 3), (2, 2), 'valid', 'relu')
    c = Conv2D_BN_AC(X, 192, (1, 1), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(c, 224, (3, 3), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(c, 256, (3, 3), (2, 2), 'valid', 'relu')
    X = concatenate([a,b,c], axis=3)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    return X

def Reduction_B(X):
    a = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(X)
    b = Conv2D_BN_AC(X, 192, (1, 1), (1, 1), 'same', 'relu')
    b = Conv2D_BN_AC(b, 192, (3, 3), (2, 2), 'valid', 'relu')
    c = Conv2D_BN_AC(X, 256, (1, 1), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(c, 256, (1, 7), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(c, 320, (7, 1), (1, 1), 'same', 'relu')
    c = Conv2D_BN_AC(c, 320, (3, 3), (2, 2), 'valid', 'relu')
    X = concatenate([a,b,c], axis=3)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    return X

def Inception_v4(input_shape = (299,299,3)):
    X_input = Input(input_shape)
    X = Stem(X_input)
    for i in np.arange(4):
        X = Inception_A(X)
    X = Reduction_A(X)
    for i in np.arange(7):
        X = Inception_B(X)
    X = Reduction_B(X)    
    for i in np.arange(3):
        X = Inception_C(X)
    X = AveragePooling2D((8,8))(X)
    X = Dropout(0.8)(X)
    X = Flatten()(X)
    X = Dense(2, activation='softmax')(X)
    model = Model(inputs = X_input, outputs = X, name='Inception_v4')
    return model


In [25]:
model = Inception_v4()

In [26]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_458 (Conv2D)             (None, 149, 149, 32) 896         input_10[0][0]                   
__________________________________________________________________________________________________
batch_normalization_510 (BatchN (None, 149, 149, 32) 128         conv2d_458[0][0]                 
__________________________________________________________________________________________________
activation_519 (Activation)     (None, 149, 149, 32) 0           batch_normalization_510[0][0]    
__________________________________________________________________________________________________
conv2d_459

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,epochs = 200, batch_size=32,validation_data=(x_test,y_test), shuffle = True)